In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from metrics import binary_evaluate
import time
import copy

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from metrics import multi_evaluate

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

In [4]:
def preprocessing(raw_data):
    df = copy.deepcopy(raw_data)
    label = 'label'
    target = df.pop(label)
    
    numerical_features = [x for x in df.columns if df[x].dtype == np.float64 or df[x].dtype == np.int64]
    categorical_features = [x for x in df.columns if df[x].dtype == object]
    
    # convert object to int
    lbe = LabelEncoder()
    for feat in categorical_features:
        df[feat] = lbe.fit_transform(df[feat])
    
    # normalize the features
    mms = MinMaxScaler()
    df[numerical_features] = mms.fit_transform(df[numerical_features])
    
    return df, target, numerical_features, categorical_features

In [5]:
train_path = './data/nslkdd/KDDTrain_five_class.csv'
raw_data = pd.read_csv(train_path)
raw_data['label'] = raw_data['types']
raw_data.drop('types', axis=1, inplace=True)
type2idx = {type: idx for idx, type in enumerate(raw_data['label'].unique())}
idx2type = {idx: type for idx, type in enumerate(raw_data['label'].unique())}
raw_data['label'] = raw_data['label'].apply(lambda x: type2idx[x])
raw_data.label = raw_data.label.astype(int)

In [6]:
features, target, numerical_features, categorical_features = preprocessing(raw_data)

In [10]:
total_details = dict()

In [11]:
cv_details = dict()
kf = KFold(n_splits=5, shuffle=True, random_state=1)
for idx, (train_idx, test_idx) in enumerate(kf.split(features, target)):
    x_train, y_train, x_test, y_test = features.loc[train_idx], target.loc[train_idx], features.loc[test_idx], target.loc[test_idx]
    lr = LogisticRegression(max_iter=100000)
    train_start_time = time.perf_counter()
    lr.fit(x_train, y_train)
    train_end_time = time.perf_counter()

    test_start_time = time.perf_counter()
    y_pred = lr.predict(x_test)
    test_end_time = time.perf_counter()

    detail = multi_evaluate(y_test, y_pred, idx2type)
    detail['training_time'] = train_end_time - train_start_time
    detail['testing_time'] = test_end_time - test_start_time
    cv_details[idx+1] = detail
total_details['lr'] = cv_details

In [12]:
cv_details = dict()
kf = KFold(n_splits=5, shuffle=True, random_state=1)
for idx, (train_idx, test_idx) in enumerate(kf.split(features, target)):
    x_train, y_train, x_test, y_test = features.loc[train_idx], target.loc[train_idx], features.loc[test_idx], target.loc[test_idx]
    cart = DecisionTreeClassifier()
    train_start_time = time.perf_counter()
    cart.fit(x_train, y_train)
    train_end_time = time.perf_counter()

    test_start_time = time.perf_counter()
    y_pred = cart.predict(x_test)
    test_end_time = time.perf_counter()

    detail = multi_evaluate(y_test, y_pred, idx2type)
    detail['training_time'] = train_end_time - train_start_time
    detail['testing_time'] = test_end_time - test_start_time
    cv_details[idx+1] = detail
total_details['CART'] = cv_details

In [13]:
cv_details = dict()
kf = KFold(n_splits=5, shuffle=True, random_state=1)
for idx, (train_idx, test_idx) in enumerate(kf.split(features, target)):
    x_train, y_train, x_test, y_test = features.loc[train_idx], target.loc[train_idx], features.loc[test_idx], target.loc[test_idx]
    c45 = DecisionTreeClassifier(criterion='entropy')
    train_start_time = time.perf_counter()
    c45.fit(x_train, y_train)
    train_end_time = time.perf_counter()

    test_start_time = time.perf_counter()
    y_pred = c45.predict(x_test)
    test_end_time = time.perf_counter()

    detail = multi_evaluate(y_test, y_pred, idx2type)
    detail['training_time'] = train_end_time - train_start_time
    detail['testing_time'] = test_end_time - test_start_time
    cv_details[idx+1] = detail
total_details['C4.5'] = cv_details

In [14]:
cv_details = dict()
kf = KFold(n_splits=5, shuffle=True, random_state=1)
for idx, (train_idx, test_idx) in enumerate(kf.split(features, target)):
    x_train, y_train, x_test, y_test = features.loc[train_idx], target.loc[train_idx], features.loc[test_idx], target.loc[test_idx]
    knn = KNeighborsClassifier()
    train_start_time = time.perf_counter()
    knn.fit(x_train, y_train)
    train_end_time = time.perf_counter()

    test_start_time = time.perf_counter()
    y_pred = knn.predict(x_test)
    test_end_time = time.perf_counter()

    detail = multi_evaluate(y_test, y_pred, idx2type)
    detail['training_time'] = train_end_time - train_start_time
    detail['testing_time'] = test_end_time - test_start_time
    cv_details[idx+1] = detail
total_details['KNN'] = cv_details

In [15]:
cv_details = dict()
kf = KFold(n_splits=5, shuffle=True, random_state=1)
for idx, (train_idx, test_idx) in enumerate(kf.split(features, target)):
    x_train, y_train, x_test, y_test = features.loc[train_idx], target.loc[train_idx], features.loc[test_idx], target.loc[test_idx]
    nb = GaussianNB()
    train_start_time = time.perf_counter()
    nb.fit(x_train, y_train)
    train_end_time = time.perf_counter()

    test_start_time = time.perf_counter()
    y_pred = nb.predict(x_test)
    test_end_time = time.perf_counter()
   
    detail = multi_evaluate(y_test, y_pred, idx2type)
    detail['training_time'] = train_end_time - train_start_time
    detail['testing_time'] = test_end_time - test_start_time
    cv_details[idx+1] = detail
total_details['NB'] = cv_details

In [16]:
cv_details = dict()
kf = KFold(n_splits=5, shuffle=True, random_state=1)
for idx, (train_idx, test_idx) in enumerate(kf.split(features, target)):
    x_train, y_train, x_test, y_test = features.loc[train_idx], target.loc[train_idx], features.loc[test_idx], target.loc[test_idx]
    rf = RandomForestClassifier()
    train_start_time = time.perf_counter()
    rf.fit(x_train, y_train)
    train_end_time = time.perf_counter()

    test_start_time = time.perf_counter()
    y_pred = rf.predict(x_test)
    test_end_time = time.perf_counter()

    detail = multi_evaluate(y_test, y_pred, idx2type)
    detail['training_time'] = train_end_time - train_start_time
    detail['testing_time'] = test_end_time - test_start_time
    cv_details[idx+1] = detail
total_details['RF'] = cv_details

In [17]:
cv_details = dict()
kf = KFold(n_splits=5, shuffle=True, random_state=1)
for idx, (train_idx, test_idx) in enumerate(kf.split(features, target)):
    x_train, y_train, x_test, y_test = features.loc[train_idx], target.loc[train_idx], features.loc[test_idx], target.loc[test_idx]
    svc = LinearSVC()
    train_start_time = time.perf_counter()
    svc.fit(x_train, y_train)
    train_end_time = time.perf_counter()

    test_start_time = time.perf_counter()
    y_pred = svc.predict(x_test)
    test_end_time = time.perf_counter()

    detail = multi_evaluate(y_test, y_pred, idx2type)
    detail['training_time'] = train_end_time - train_start_time
    detail['testing_time'] = test_end_time - test_start_time
    cv_details[idx+1] = detail
total_details['SVC'] = cv_details

/home/hadoop/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/hadoop/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/hadoop/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/hadoop/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/hadoop/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/svm/_base.py:985: Con

In [23]:
result = pd.DataFrame.from_dict({(i, j, k): total_details[i][j][k] for i in total_details.keys() for j in total_details[i].keys() for k in total_details[i][j].keys()}).T


In [25]:
result.index.names = ['learner', 'cv', 'type']

In [29]:
result.to_csv('./result/nslkdd/five/baseline.csv')